### Deep Neural Network in TensorFlow
![title](deep.jpg)

### Import the MNIST Dataset

* Import the Dataset using Keras
* Reshape the data
* One-Hot Encoder on the labels

In [15]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import math

# Import MNIST data
# (x_train, y_train), (x_test, y_test)
(train_features, train_labels), (test_features, test_labels) = tf.keras.datasets.mnist.load_data()


n_input = 784  # MNIST data input (img shape: 28*28)
n_classes = 10  # MNIST total classes (0-9 digits)
                                                               # Dimensions
train_features = train_features.astype('float32') / 255
test_features = test_features.astype('float32') / 255
train_labels = train_labels.astype('float32') / 255            
test_labels = test_labels.astype('float32') / 255
train_labels = tf.keras.utils.to_categorical(train_labels, 10) # 60000 * 10
test_labels = tf.keras.utils.to_categorical(test_labels, 10)   # 10000 * 10


def batches(batch_size, features, labels):
    """
    Create batches of features and labels
    :param batch_size: The batch size
    :param features: List of features
    :param labels: List of labels
    :return: Batches of (Features, Labels)
    """
    assert len(features) == len(labels)
    outout_batches = []
    out_features = []
    out_labels = []
    
    sample_size = len(features)
    for start_i in range(0, sample_size, batch_size):
        end_i = start_i + batch_size
        tmp_feat = features[start_i:end_i]
        tmp_lab = labels[start_i:end_i]
        batch = [tmp_feat, tmp_lab]
        out_features.append(tmp_feat)
        out_labels.append(tmp_lab)
        outout_batches.append(batch)
        
    return outout_batches

### Network Architecture

In [16]:
# Parameters
learning_rate = 0.001
training_epochs = 20
batch_size = 128  # Decrease batch size if you don't have enough memory
display_step = 1
n_hidden_layer = 256 # layer number of features

# Store layers weight & bias
weights = {
    'hidden_layer': tf.Variable(tf.random_normal([n_input, n_hidden_layer])),
    'out': tf.Variable(tf.random_normal([n_hidden_layer, n_classes]))
}
biases = {
    'hidden_layer': tf.Variable(tf.random_normal([n_hidden_layer])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# tf Graph input
x = tf.placeholder("float", [None, 28, 28])
y = tf.placeholder("float", [None, n_classes])

x_flat = tf.reshape(x, [-1, n_input])

# Hidden layer with RELU activation
layer_1 = tf.add(tf.matmul(x_flat, weights['hidden_layer']), biases['hidden_layer'])
layer_1 = tf.nn.relu(layer_1)

# Output layer with linear activation
logits = tf.matmul(layer_1, weights['out']) + biases['out']

### Cost Function and Optimizer

In [17]:
# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

### Training 

In [19]:
# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    # Training cycle
    for epoch in range(training_epochs):
        # Loop over all batches
        for batch_features, batch_labels in batches(batch_size, train_features, train_labels):
            sess.run(optimizer, feed_dict={x: batch_features, y: batch_labels})
        # Display logs per epoch step
        if epoch % display_step == 0:
            c = sess.run(cost, feed_dict={x: batch_features, y: batch_labels})
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(c))
    print("Optimization Finished!")

    # Test model
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    # Decrease test_size if you don't have enough memory
    test_size = 256
    print("Accuracy:", accuracy.eval({x: test_features[:test_size], y: test_labels[:test_size]}))

Epoch: 0001 cost= 0.793775618
Epoch: 0002 cost= 0.407656878
Epoch: 0003 cost= 0.220782578
Epoch: 0004 cost= 0.101185583
Epoch: 0005 cost= 0.028011193
Epoch: 0006 cost= 0.002310515
Epoch: 0007 cost= 0.000174590
Epoch: 0008 cost= 0.000022464
Epoch: 0009 cost= 0.000003981
Epoch: 0010 cost= 0.000000887
Epoch: 0011 cost= 0.000000236
Epoch: 0012 cost= 0.000000072
Epoch: 0013 cost= 0.000000024
Epoch: 0014 cost= 0.000000009
Epoch: 0015 cost= 0.000000004
Epoch: 0016 cost= 0.000000001
Epoch: 0017 cost= 0.000000001
Epoch: 0018 cost= 0.000000000
Epoch: 0019 cost= 0.000000000
Epoch: 0020 cost= 0.000000000
Optimization Finished!
Accuracy: 1.0
